In [13]:
import cv2
import mediapipe as mp
import pyautogui  # Use pyautogui for simulating media key presses
import time

# Initialize Mediapipe for hand tracking
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Helper function to check if only index finger is extended
def is_only_index_extended(landmarks):
    index_extended = landmarks[8].y < landmarks[6].y  # Index finger extended
    middle_closed = landmarks[12].y < landmarks[10].y  # Middle finger closed
    ring_closed = landmarks[16].y > landmarks[14].y    # Ring finger closed
    pinky_closed = landmarks[20].y > landmarks[18].y   # Pinky finger closed
    return index_extended and middle_closed and ring_closed and pinky_closed

# Function to check if hand is relatively still for a certain duration
def is_hand_still(current_position, last_position, last_movement_time, still_threshold=0.01, still_duration=0.15):
    distance = ((current_position[0] - last_position[0]) ** 2 + (current_position[1] - last_position[1]) ** 2) ** 0.5
    current_time = time.time()
    if distance < still_threshold:
        if current_time - last_movement_time >= still_duration:
            return True, current_time  # Hand is still, update last movement time
    else:
        last_movement_time = current_time  # Hand moved, reset timer
    return False, last_movement_time

# Swipe detection logic
def detect_swipe(previous_position, current_position, last_swipe_time, swipe_threshold=0.04):
    distance_moved = abs(current_position[0] - previous_position[0])
    current_time = time.time()
    if distance_moved > swipe_threshold and current_time - last_swipe_time > 0.7:
        swipe_direction = 'left' if current_position[0] < previous_position[0] else 'right'
        last_swipe_time = current_time
        return swipe_direction, last_swipe_time
    return None, last_swipe_time

def is_fist_closed(landmarks):
    return (landmarks[8].y > landmarks[6].y and  # Index finger curled
            landmarks[12].y > landmarks[10].y and  # Middle finger curled
            landmarks[16].y > landmarks[14].y and  # Ring finger curled
            landmarks[20].y > landmarks[18].y)  # Pinky finger curled

def is_palm_open(landmarks):
    return (landmarks[8].y < landmarks[6].y and  # Index finger extended
            landmarks[12].y < landmarks[10].y and  # Middle finger extended
            landmarks[16].y < landmarks[14].y and  # Ring finger extended
            landmarks[20].y < landmarks[18].y)  # Pinky finger extended

cap = cv2.VideoCapture("http://192.168.137.49:8080/video")
last_swipe_time = 0
previous_position = None
last_movement_time = time.time()  # To track when the hand was last still
hand_ready_for_swipe = False      # Flag to indicate readiness for swipe
# Initialize additional variables for play/pause detection
last_fist_time = time.time()
last_palm_time = time.time()
fist_detected = False
palm_detected = False
hand_ready_for_gesture = False  # New flag to track readiness for play/pause gestures

# Main loop for gesture detection and control
while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            index_finger_tip = hand_landmarks.landmark[12]
            current_position = (index_finger_tip.x, index_finger_tip.y)

            if previous_position is None:
                previous_position = current_position

            # Check if only the index finger is extended for swipe detection
            if is_only_index_extended(hand_landmarks.landmark):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                if hand_still:
                    hand_ready_for_swipe = True
                if hand_ready_for_swipe:
                    swipe_direction, last_swipe_time = detect_swipe(previous_position, current_position, last_swipe_time)
                    if swipe_direction == 'left':
                        print("Swipe left detected: Previous track")
                        pyautogui.press("prevtrack")
                        hand_ready_for_swipe = False
                        previous_position = current_position
                    elif swipe_direction == 'right':
                        print("Swipe right detected: Next track")
                        pyautogui.press("nexttrack")
                        hand_ready_for_swipe = False
                        previous_position = current_position

            # Check if fist is closed and hand is still for pause action
            if is_fist_closed(hand_landmarks.landmark):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
                if hand_still:
                    hand_ready_for_gesture = True
                if hand_ready_for_gesture and not fist_detected:
                    last_fist_time = time.time()
                    fist_detected = True
                elif fist_detected and time.time() - last_fist_time > 1:
                    print("Fist closed and still for 1 second: Pausing")
                    pyautogui.press("playpause")
                    hand_ready_for_gesture = False
                    fist_detected = False

            else:
                fist_detected = False  # Reset if hand is no longer a fist

            # Check if palm is open and hand is still for play action
            if is_palm_open(hand_landmarks.landmark):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
                if hand_still:
                    hand_ready_for_gesture = True
                if hand_ready_for_gesture and not palm_detected:
                    last_palm_time = time.time()
                    palm_detected = True
                elif palm_detected and time.time() - last_palm_time > 1:
                    print("Open palm and still for 1 second: Playing")
                    pyautogui.press("playpause")
                    hand_ready_for_gesture = False
                    palm_detected = False

            else:
                palm_detected = False

            # Update previous_position for the next frame
            previous_position = current_position
    elif hand_ready_for_swipe or hand_ready_for_gesture:
        hand_ready_for_swipe = False
        hand_ready_for_gesture = False

    # Display the video feed with hand landmarks
    cv2.imshow('Hand Gesture Control', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


Open palm and still for 1 second: Playing
Fist closed and still for 1 second: Pausing
Fist closed and still for 1 second: Pausing
Fist closed and still for 1 second: Pausing
Fist closed and still for 1 second: Pausing
Open palm and still for 1 second: Playing
Open palm and still for 1 second: Playing
Fist closed and still for 1 second: Pausing
Fist closed and still for 1 second: Pausing
Open palm and still for 1 second: Playing


In [ ]:
import cv2
import mediapipe as mp
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
import time

# Initialize Mediapipe for hand tracking
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Pycaw setup for audio control (remove if not needed)
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Helper function to check if only index finger is extended
def is_only_index_extended(landmarks):
    # Check distances between each finger's tip and its corresponding pip joint
    index_extended = landmarks[8].y < landmarks[6].y  # Index finger extended
    middle_closed = landmarks[12].y < landmarks[10].y  # Middle finger closed
    ring_closed = landmarks[16].y > landmarks[14].y    # Ring finger closed
    pinky_closed = landmarks[20].y > landmarks[18].y   # Pinky finger closed
    
    return index_extended and middle_closed and ring_closed and pinky_closed

# Function to check if hand is relatively still for a certain duration
def is_hand_still(current_position, last_position, last_movement_time, still_threshold=0.01, still_duration=0.15):
    # Calculate the Euclidean distance between the current and last position
    distance = ((current_position[0] - last_position[0]) ** 2 + (current_position[1] - last_position[1]) ** 2) ** 0.5
    
    # Get current time
    current_time = time.time()

    # Check if movement is within the threshold and duration has passed
    if distance < still_threshold:
        if current_time - last_movement_time >= still_duration:
            return True, current_time  # Hand is still, update last movement time
    else:
        last_movement_time = current_time  # Hand moved, reset timer

    return False, last_movement_time

# Swipe detection logic
def detect_swipe(previous_position, current_position, last_swipe_time, swipe_threshold=0.04):
    # Calculate the horizontal distance moved by the index finger
    distance_moved = abs(current_position[0] - previous_position[0])

    # Detect swipe based on speed and direction
    current_time = time.time()
    if distance_moved > swipe_threshold and current_time - last_swipe_time > 0.7:
        swipe_direction = 'left' if current_position[0] < previous_position[0] else 'right'
        last_swipe_time = current_time
        return swipe_direction, last_swipe_time
    return None, last_swipe_time

def is_fist_closed(landmarks):
    return (landmarks[8].y > landmarks[6].y and  # Index finger curled
            landmarks[12].y > landmarks[10].y and  # Middle finger curled
            landmarks[16].y > landmarks[14].y and  # Ring finger curled
            landmarks[20].y > landmarks[18].y)  # Pinky finger curled

# Helper function to check if all fingers are extended (open palm)
def is_palm_open(landmarks):
    return (landmarks[8].y < landmarks[6].y and  # Index finger extended
            landmarks[12].y < landmarks[10].y and  # Middle finger extended
            landmarks[16].y < landmarks[14].y and  # Ring finger extended
            landmarks[20].y < landmarks[18].y)  # Pinky finger extended

# Access the webcam
cap = cv2.VideoCapture("http://192.168.137.49:8080/video")
last_swipe_time = 0
previous_position = None
last_movement_time = time.time()  # To track when the hand was last still
hand_ready_for_swipe = False      # Flag to indicate readiness for swipe
# Initialize additional variables for play/pause detection
last_fist_time = time.time()
last_palm_time = time.time()
fist_detected = False
palm_detected = False
hand_ready_for_gesture = False  # New flag to track readiness for play/pause gestures

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Get the index finger tip position (landmark 8)
            index_finger_tip = hand_landmarks.landmark[12]
            current_position = (index_finger_tip.x, index_finger_tip.y)

            if previous_position is None:
                previous_position = current_position

            # Check if only the index finger is extended for swipe detection
            if is_only_index_extended(hand_landmarks.landmark):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                if hand_still:
                    hand_ready_for_swipe = True
                if hand_ready_for_swipe:
                    swipe_direction, last_swipe_time = detect_swipe(previous_position, current_position, last_swipe_time)
                    if swipe_direction == 'left':
                        print("Swipe left detected: Previous track")
                        hand_ready_for_swipe = False
                        previous_position = current_position
                    elif swipe_direction == 'right':
                        print("Swipe right detected: Next track")
                        hand_ready_for_swipe = False
                        previous_position = current_position

            # Check if fist is closed and hand is still for pause action
            if is_fist_closed(hand_landmarks.landmark):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
                if hand_still:
                    hand_ready_for_gesture = True  # Mark readiness if hand is still
                if hand_ready_for_gesture and not fist_detected:
                    last_fist_time = time.time()
                    fist_detected = True
                elif fist_detected and time.time() - last_fist_time > 1:
                    print("Fist closed and still for 1 second: Pausing")
                    hand_ready_for_gesture = False  # Reset after action
                    fist_detected = False
                    # Implement pause action here

            else:
                fist_detected = False  # Reset if hand is no longer a fist

            # Check if palm is open and hand is still for play action
            if is_palm_open(hand_landmarks.landmark):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
                if hand_still:
                    hand_ready_for_gesture = True  # Mark readiness if hand is still
                if hand_ready_for_gesture and not palm_detected:
                    last_palm_time = time.time()
                    palm_detected = True
                elif palm_detected and time.time() - last_palm_time > 1:
                    print("Open palm and still for 1 second: Playing")
                    hand_ready_for_gesture = False  # Reset after action
                    palm_detected = False
                    # Implement play action here

            else:
                palm_detected = False  # Reset if hand is no longer open palm

            # Update previous_position for the next frame
            previous_position = current_position
    elif hand_ready_for_swipe or hand_ready_for_gesture:
        hand_ready_for_swipe = False
        hand_ready_for_gesture = False

    cv2.imshow('Hand Gesture Control', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



In [3]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import cv2
import mediapipe as mp
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
import time

# Initialize Mediapipe for hand tracking
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Pycaw setup for audio control (remove if not needed)
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Helper function to check if only index finger is extended
def is_only_index_extended(landmarks):
    # Check distances between each finger's tip and its corresponding pip joint
    index_extended = landmarks[8].y < landmarks[6].y  # Index finger extended
    middle_closed = landmarks[12].y < landmarks[10].y  # Middle finger closed
    ring_closed = landmarks[16].y > landmarks[14].y    # Ring finger closed
    pinky_closed = landmarks[20].y > landmarks[18].y   # Pinky finger closed
    
    return index_extended and middle_closed and ring_closed and pinky_closed

# Function to check if hand is relatively still for a certain duration
def is_hand_still(current_position, last_position, last_movement_time, still_threshold=0.01, still_duration=0.15):
    # Calculate the Euclidean distance between the current and last position
    distance = ((current_position[0] - last_position[0]) ** 2 + (current_position[1] - last_position[1]) ** 2) ** 0.5
    
    # Get current time
    current_time = time.time()

    # Check if movement is within the threshold and duration has passed
    if distance < still_threshold:
        if current_time - last_movement_time >= still_duration:
            return True, current_time  # Hand is still, update last movement time
    else:
        last_movement_time = current_time  # Hand moved, reset timer

    return False, last_movement_time

# Swipe detection logic
def detect_swipe(previous_position, current_position, last_swipe_time, swipe_threshold=0.04):
    # Calculate the horizontal distance moved by the index finger
    distance_moved = abs(current_position[0] - previous_position[0])

    # Detect swipe based on speed and direction
    current_time = time.time()
    if distance_moved > swipe_threshold and current_time - last_swipe_time > 0.7:
        swipe_direction = 'left' if current_position[0] < previous_position[0] else 'right'
        last_swipe_time = current_time
        return swipe_direction, last_swipe_time
    return None, last_swipe_time

def is_fist_closed(landmarks):
    return (landmarks[8].y > landmarks[6].y and  # Index finger curled
            landmarks[12].y > landmarks[10].y and  # Middle finger curled
            landmarks[16].y > landmarks[14].y and  # Ring finger curled
            landmarks[20].y > landmarks[18].y)  # Pinky finger curled

# Helper function to check if all fingers are extended (open palm)
def is_palm_open(landmarks):
    return (landmarks[8].y < landmarks[6].y and  # Index finger extended
            landmarks[12].y < landmarks[10].y and  # Middle finger extended
            landmarks[16].y < landmarks[14].y and  # Ring finger extended
            landmarks[20].y < landmarks[18].y)  # Pinky finger extended

# Spotify Authentication Setup
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="7b6e2dbc5d114cfe89edc5aeb3689925",
                                               client_secret="b9aef05ebdeb43f3bbb4b16ee0a3a183",
                                               redirect_uri="http://localhost:8888/callback",
                                               scope="user-modify-playback-state user-read-playback-state"))

try:
    playback = sp.current_playback()
    if playback:
        print("Spotify connection successful. Playback state:", playback)
    else:
        print("Spotify connected, but no active playback device.")
except Exception as e:
    print("Error with Spotify API:", e)


# Function to control Spotify playback
def play_spotify():
    sp.start_playback(device_id='76ae3d4c59b12e6035b0082c51fbda1b78b3ef1e')

def pause_spotify():
    sp.pause_playback(device_id='76ae3d4c59b12e6035b0082c51fbda1b78b3ef1e')

def next_track():
    sp.next_track(device_id='76ae3d4c59b12e6035b0082c51fbda1b78b3ef1e')

def previous_track():
    sp.previous_track(device_id='76ae3d4c59b12e6035b0082c51fbda1b78b3ef1e')


# cap = cv2.VideoCapture("http://192.168.137.102:8080/video")
# last_swipe_time = 0
# previous_position = None
# last_movement_time = time.time()  # To track when the hand was last still
# hand_ready_for_swipe = False      # Flag to indicate readiness for swipe
# # Initialize additional variables for play/pause detection
# last_fist_time = time.time()
# last_palm_time = time.time()
# fist_detected = False
# palm_detected = False
# hand_ready_for_gesture = False  # New flag to track readiness for play/pause gestures

# # Update existing code in the main loop where gestures are detected:
# while cap.isOpened():
#     success, image = cap.read()
#     if not success:
#         print("Ignoring empty camera frame.")
#         continue

#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     results = hands.process(image)
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#     if results.multi_hand_landmarks:
#         for hand_landmarks in results.multi_hand_landmarks:
#             mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
#             # Get the index finger tip position (landmark 8)
#             index_finger_tip = hand_landmarks.landmark[12]
#             current_position = (index_finger_tip.x, index_finger_tip.y)

#             if previous_position is None:
#                 previous_position = current_position

#             # Check if only the index finger is extended for swipe detection
#             if is_only_index_extended(hand_landmarks.landmark):
#                 hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
#                 if hand_still:
#                     hand_ready_for_swipe = True
#                 if hand_ready_for_swipe:
#                     swipe_direction, last_swipe_time = detect_swipe(previous_position, current_position, last_swipe_time)
#                     if swipe_direction == 'left':
#                         print("Swipe left detected: Previous track")
#                         previous_track()
#                         hand_ready_for_swipe = False
#                         previous_position = current_position
#                     elif swipe_direction == 'right':
#                         print("Swipe right detected: Next track")
#                         next_track()
#                         hand_ready_for_swipe = False
#                         previous_position = current_position

#             # Check if fist is closed and hand is still for pause action
#             if is_fist_closed(hand_landmarks.landmark):
#                 hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
#                 if hand_still:
#                     hand_ready_for_gesture = True  # Mark readiness if hand is still
#                 if hand_ready_for_gesture and not fist_detected:
#                     last_fist_time = time.time()
#                     fist_detected = True
#                 elif fist_detected and time.time() - last_fist_time > 1:
#                     print("Fist closed and still for 1 second: Pausing")
#                     pause_spotify()
#                     hand_ready_for_gesture = False  # Reset after action
#                     fist_detected = False
#                     # Implement pause action here

#             else:
#                 fist_detected = False  # Reset if hand is no longer a fist

#             # Check if palm is open and hand is still for play action
#             if is_palm_open(hand_landmarks.landmark):
#                 hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
#                 if hand_still:
#                     hand_ready_for_gesture = True  # Mark readiness if hand is still
#                 if hand_ready_for_gesture and not palm_detected:
#                     last_palm_time = time.time()
#                     palm_detected = True
#                 elif palm_detected and time.time() - last_palm_time > 1:
#                     print("Open palm and still for 1 second: Playing")
#                     play_spotify()
#                     hand_ready_for_gesture = False  # Reset after action
#                     palm_detected = False
#                     # Implement play action here

#             else:
#                 palm_detected = False  # Reset if hand is no longer open palm

#             # Update previous_position for the next frame
#             previous_position = current_position
#     elif hand_ready_for_swipe or hand_ready_for_gesture:
#         hand_ready_for_swipe = False
#         hand_ready_for_gesture = False

#     # Display the video feed with hand landmarks
#     cv2.imshow('Hand Gesture Control', image)
#     if cv2.waitKey(5) & 0xFF == 27:
#         break

# cap.release()
# cv2.destroyAllWindows()


Spotify connection successful. Playback state: {'device': {'id': 'ba8b108e56582fa903e6135307113a1c7ec59b5a', 'is_active': True, 'is_private_session': False, 'is_restricted': False, 'name': 'Error 2.0', 'supports_volume': False, 'type': 'Smartphone', 'volume_percent': 100}, 'shuffle_state': True, 'smart_shuffle': False, 'repeat_state': 'off', 'timestamp': 1731516231641, 'context': {'external_urls': {'spotify': 'https://open.spotify.com/playlist/7cl0xFp2x8Xp8HevR4kigw'}, 'href': 'https://api.spotify.com/v1/playlists/7cl0xFp2x8Xp8HevR4kigw', 'type': 'playlist', 'uri': 'spotify:playlist:7cl0xFp2x8Xp8HevR4kigw'}, 'progress_ms': 51452, 'item': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4PULA4EFzYTrxYvOVlwpiQ'}, 'href': 'https://api.spotify.com/v1/artists/4PULA4EFzYTrxYvOVlwpiQ', 'id': '4PULA4EFzYTrxYvOVlwpiQ', 'name': 'Sidhu Moose Wala', 'type': 'artist', 'uri': 'spotify:artist:4PULA4EFzYTrxYvOVlwpiQ'}], 'available_markets': [

In [2]:
sp.pause_playback(device_id='ba8b108e56582fa903e6135307113a1c7ec59b5a')
# sp.pause_playback()

HTTP Error for PUT to https://api.spotify.com/v1/player/pause?device_id=ba8b108e56582fa903e6135307113a1c7ec59b5a with Params: {} returned 404 due to Service not found


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/player/pause?device_id=ba8b108e56582fa903e6135307113a1c7ec59b5a:
 Service not found, reason: None

In [ ]:
from selenium.webdriver import Chrome,ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time

options = ChromeOptions()

options.add_argument(r"user-data-dir=C:\Users\abhay\Desktop\CG (CMP513, CMP514)\Project\browser")
options.add_argument("--headless")

browser = Chrome(options=options)
wait = WebDriverWait(browser, 3)


browser.get("https://open.spotify.com/")

try:
    if wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "ButtonInner-sc-14ud5tc-0.kwEVAz.encore-inverted-light-set"))).text == "Log in":
        logged_in = False
except:
    logged_in = True

if not logged_in:
    print("User is not logged in. Restarting in non-headless mode.")
    browser.quit()

    options = ChromeOptions()
    options.add_argument(r"user-data-dir=C:\Users\abhay\Desktop\CG (CMP513, CMP514)\Project\browser")

    browser = Chrome(options=options)
    browser.get(r"https://accounts.spotify.com/en/login")

    br = True
    while(br):
        try: 
            for text in browser.find_elements(By.CLASS_NAME, "encore-text.encore-text-body-medium-bold.sc-iKTcqh.doOTMr"):
                if text.text == "Web Player":
                    # text.click()
                    br = False
        except:
            time.sleep(2)
    browser.quit()

    options = ChromeOptions()
    options.add_argument(r"user-data-dir=C:\Users\abhay\Desktop\CG (CMP513, CMP514)\Project\browser")
    options.add_argument("--headless")

    browser = Chrome(options=options)
    wait = WebDriverWait(browser, 10)
    browser.get(r"https://open.spotify.com/")
else:
    print("User is already logged in. Continuing in headless mode.")

SessionNotCreatedException: Message: session not created: Chrome failed to start: crashed.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location C:\Program Files\Google\Chrome\Application\chrome.exe is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
	GetHandleVerifier [0x00007FF7D77A38A5+3004357]
	(No symbol) [0x00007FF7D7439970]
	(No symbol) [0x00007FF7D72E582A]
	(No symbol) [0x00007FF7D731CDA7]
	(No symbol) [0x00007FF7D7318DBE]
	(No symbol) [0x00007FF7D7366E01]
	(No symbol) [0x00007FF7D7366480]
	(No symbol) [0x00007FF7D735B983]
	(No symbol) [0x00007FF7D7327628]
	(No symbol) [0x00007FF7D7328791]
	GetHandleVerifier [0x00007FF7D77CA00D+3161901]
	GetHandleVerifier [0x00007FF7D781E060+3506048]
	GetHandleVerifier [0x00007FF7D781400D+3465005]
	GetHandleVerifier [0x00007FF7D7590EEB+830987]
	(No symbol) [0x00007FF7D744467F]
	(No symbol) [0x00007FF7D74409D4]
	(No symbol) [0x00007FF7D7440B6D]
	(No symbol) 

In [1]:
import cv2
import mediapipe as mp
import time

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

def is_only_index_extended(landmarks):
    index_extended = landmarks[8].y < landmarks[6].y 
    middle_closed = landmarks[12].y < landmarks[10].y
    ring_closed = landmarks[16].y > landmarks[14].y 
    pinky_closed = landmarks[20].y > landmarks[18].y
    
    return index_extended and middle_closed and ring_closed and pinky_closed

def is_hand_still(current_position, last_position, last_movement_time, still_threshold=0.01, still_duration=0.15):
    distance = ((current_position[0] - last_position[0]) ** 2 + (current_position[1] - last_position[1]) ** 2) ** 0.5
    
    current_time = time.time()

    if distance < still_threshold:
        if current_time - last_movement_time >= still_duration:
            return True, current_time
    else:
        last_movement_time = current_time

    return False, last_movement_time

def detect_swipe(previous_position, current_position, last_swipe_time, swipe_threshold=0.04):
    distance_moved = abs(current_position[0] - previous_position[0])

    current_time = time.time()
    if distance_moved > swipe_threshold and current_time - last_swipe_time > 0.7:
        swipe_direction = 'left' if current_position[0] < previous_position[0] else 'right'
        last_swipe_time = current_time
        return swipe_direction, last_swipe_time
    return None, last_swipe_time

def is_fist_closed(landmarks):
    return (landmarks[8].y > landmarks[6].y and
            landmarks[12].y > landmarks[10].y and
            landmarks[16].y > landmarks[14].y and
            landmarks[20].y > landmarks[18].y)

def is_palm_open(landmarks):
    return (landmarks[8].y < landmarks[6].y and
            landmarks[12].y < landmarks[10].y and
            landmarks[16].y < landmarks[14].y and
            landmarks[20].y < landmarks[18].y)

def send_command(command):
    main_hub = browser.find_element(By.CLASS_NAME, "yglmI5m3fCc8baD1Kwdw")
    try:
        main_hub.find_element(By.CLASS_NAME, "gQoa8JTSpjSmYyABcag2.encore-bright-accent-set.T3hkVxXuSbCYOD2GIeQd")
    except:
        print("No Devices Available")
        return
    
    if command == "play":
        if main_hub.find_element(By.CLASS_NAME, "Button-sc-qlcn5g-0.iJUiBm").get_attribute("aria-label") == "Play":
            main_hub.find_element(By.CLASS_NAME, "Button-sc-qlcn5g-0.iJUiBm").click()
            return
        else:
            print("Device is already playing.")
    
    elif command == "pause":
        if main_hub.find_element(By.CLASS_NAME, "Button-sc-qlcn5g-0.iJUiBm").get_attribute("aria-label") == "Pause":
            main_hub.find_element(By.CLASS_NAME, "Button-sc-qlcn5g-0.iJUiBm").click()
            return
        else:
            print("Device is already playing.")

    elif command == "next_track":
        side_hub = main_hub.find_element(By.CLASS_NAME, "Qt226Z4rBQs53aedRQBQ")
        for button in side_hub.find_elements(By.CLASS_NAME, "Button-sc-1dqy6lx-0.dmdXQN"):
            if button.get_attribute("data-testid") == "control-button-skip-forward":
                button.click()
                return

    elif command == "prev_track":
        song_name = browser.find_element(By.XPATH, "/html/body/div[4]/div/div[2]/div[3]/footer/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/span/a").text
        side_hub = main_hub.find_element(By.CLASS_NAME, "NKUrT1GciYXAEEUtagN1")
        for button in side_hub.find_elements(By.CLASS_NAME, "Button-sc-1dqy6lx-0.dmdXQN"):
            if button.get_attribute("data-testid") == "control-button-skip-back":
                button.click()
                if song_name == browser.find_element(By.XPATH, "/html/body/div[4]/div/div[2]/div[3]/footer/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/span/a").text:
                    time.sleep(1)
                    button.click()

def previous_track():
    send_command("prev_track")
def next_track():
    send_command("next_track")
def pause_spotify():
    send_command("pause")
def play_spotify():
    send_command("play")


cap = cv2.VideoCapture("http://192.168.2.247:8080/video")
last_swipe_time = 0
previous_position = None
last_movement_time = time.time()
hand_ready_for_swipe = False
last_fist_time = time.time()
last_palm_time = time.time()
fist_detected = False
palm_detected = False
hand_ready_for_gesture = False

song_name = browser.find_element(By.XPATH, "/html/body/div[4]/div/div[2]/div[3]/footer/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/span/a").text


while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            index_finger_tip = hand_landmarks.landmark[12]
            current_position = (index_finger_tip.x, index_finger_tip.y)

            if previous_position is None:
                previous_position = current_position

            if is_only_index_extended(hand_landmarks.landmark):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                if hand_still:
                    hand_ready_for_swipe = True
                if hand_ready_for_swipe:
                    swipe_direction, last_swipe_time = detect_swipe(previous_position, current_position, last_swipe_time)
                    if swipe_direction == 'left':
                        print("Swipe left detected: Previous track")
                        previous_track()
                        hand_ready_for_swipe = False
                        previous_position = current_position
                    elif swipe_direction == 'right':
                        print("Swipe right detected: Next track")
                        next_track()
                        hand_ready_for_swipe = False
                        previous_position = current_position

            if is_fist_closed(hand_landmarks.landmark):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
                if hand_still:
                    hand_ready_for_gesture = True
                if hand_ready_for_gesture and not fist_detected:
                    last_fist_time = time.time()
                    fist_detected = True
                elif fist_detected and time.time() - last_fist_time > 1:
                    print("Fist closed and still for 1 second: Pausing")
                    pause_spotify()
                    hand_ready_for_gesture = False
                    fist_detected = False

            else:
                fist_detected = False

            if is_palm_open(hand_landmarks.landmark):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
                if hand_still:
                    hand_ready_for_gesture = True
                if hand_ready_for_gesture and not palm_detected:
                    last_palm_time = time.time()
                    palm_detected = True
                elif palm_detected and time.time() - last_palm_time > 1:
                    print("Open palm and still for 1 second: Playing")
                    play_spotify()
                    hand_ready_for_gesture = False 
                    palm_detected = False

            else:
                palm_detected = False

            previous_position = current_position
    elif hand_ready_for_swipe or hand_ready_for_gesture:
        hand_ready_for_swipe = False
        hand_ready_for_gesture = False

    cv2.imshow('Hand Gesture Control', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'browser' is not defined

In [ ]:
pip install selenium

In [ ]:
pip install numpy==1.23.5

In [ ]:
pip install --upgrade protobuf==3.20.0

In [ ]:
pip install tensorflow-gpu==2.10.0


In [ ]:
pip install mediapipe==0.10.0

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
print("GPU available:", tf.config.list_physical_devices('GPU'))

In [ ]:
import cv2
import mediapipe as mp
import time
import tkinter as tk
from tkinter import Label
from PIL import Image, ImageTk

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

def is_only_index_extended(landmarks):
    index_extended = landmarks[8].y < landmarks[6].y 
    middle_closed = landmarks[12].y < landmarks[10].y
    ring_closed = landmarks[16].y > landmarks[14].y 
    pinky_closed = landmarks[20].y > landmarks[18].y
    
    return index_extended and middle_closed and ring_closed and pinky_closed

def is_hand_still(current_position, last_position, last_movement_time, still_threshold=0.01, still_duration=0.15):
    distance = ((current_position[0] - last_position[0]) ** 2 + (current_position[1] - last_position[1]) ** 2) ** 0.5
    
    current_time = time.time()

    if distance < still_threshold:
        if current_time - last_movement_time >= still_duration:
            return True, current_time
    else:
        last_movement_time = current_time

    return False, last_movement_time

def detect_swipe(previous_position, current_position, last_swipe_time, swipe_threshold=0.04):
    distance_moved = abs(current_position[0] - previous_position[0])

    current_time = time.time()
    if distance_moved > swipe_threshold and current_time - last_swipe_time > 0.7:
        swipe_direction = 'left' if current_position[0] < previous_position[0] else 'right'
        last_swipe_time = current_time
        return swipe_direction, last_swipe_time
    return None, last_swipe_time

def is_fist_closed(landmarks):
    return (landmarks[8].y > landmarks[6].y and
            landmarks[12].y > landmarks[10].y and
            landmarks[16].y > landmarks[14].y and
            landmarks[20].y > landmarks[18].y)

def is_palm_open(landmarks):
    return (landmarks[8].y < landmarks[6].y and
            landmarks[12].y < landmarks[10].y and
            landmarks[16].y < landmarks[14].y and
            landmarks[20].y < landmarks[18].y)

def send_command(command):
    main_hub = browser.find_element(By.CLASS_NAME, "yglmI5m3fCc8baD1Kwdw")
    try:
        main_hub.find_element(By.CLASS_NAME, "gQoa8JTSpjSmYyABcag2.encore-bright-accent-set.T3hkVxXuSbCYOD2GIeQd")
    except:
        print("No Devices Available")
        return
    
    if command == "play":
        if main_hub.find_element(By.CLASS_NAME, "Button-sc-qlcn5g-0.iJUiBm").get_attribute("aria-label") == "Play":
            main_hub.find_element(By.CLASS_NAME, "Button-sc-qlcn5g-0.iJUiBm").click()
            return
        else:
            print("Device is already playing.")
    
    elif command == "pause":
        if main_hub.find_element(By.CLASS_NAME, "Button-sc-qlcn5g-0.iJUiBm").get_attribute("aria-label") == "Pause":
            main_hub.find_element(By.CLASS_NAME, "Button-sc-qlcn5g-0.iJUiBm").click()
            return
        else:
            print("Device is already playing.")

    elif command == "next_track":
        side_hub = main_hub.find_element(By.CLASS_NAME, "Qt226Z4rBQs53aedRQBQ")
        for button in side_hub.find_elements(By.CLASS_NAME, "Button-sc-1dqy6lx-0.dmdXQN"):
            if button.get_attribute("data-testid") == "control-button-skip-forward":
                button.click()
                return

    elif command == "prev_track":
        song_name = browser.find_element(By.XPATH, "/html/body/div[4]/div/div[2]/div[3]/footer/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/span/a").text
        side_hub = main_hub.find_element(By.CLASS_NAME, "NKUrT1GciYXAEEUtagN1")
        for button in side_hub.find_elements(By.CLASS_NAME, "Button-sc-1dqy6lx-0.dmdXQN"):
            if button.get_attribute("data-testid") == "control-button-skip-back":
                button.click()
                if song_name == browser.find_element(By.XPATH, "/html/body/div[4]/div/div[2]/div[3]/footer/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/span/a").text:
                    time.sleep(1)
                    button.click()

def previous_track():
    display_command("Swipe detected: Playing Next Track")
    send_command("prev_track")
def next_track():
    display_command("Swipe detected: Playing Previous Track")
    send_command("next_track")
def pause_spotify():
    display_command("Fist detected: Pausing")
    send_command("pause")
def play_spotify():
    display_command("Palm detected: Playing")
    send_command("play")

cap = cv2.VideoCapture("http://192.168.2.247:8080/video")
last_swipe_time = 0
previous_position = None
last_movement_time = time.time()
hand_ready_for_swipe = False
last_fist_time = time.time()
last_palm_time = time.time()
fist_detected = False
palm_detected = False
hand_ready_for_gesture = False

command_text = ""
last_command_time = 0

# Tkinter setup
window = tk.Tk()
window.title("Hand Gesture Music Control")
window.geometry("800x600")

camera_frame = Label(window)
camera_frame.pack(side="left", padx=10, pady=10)

song_name_label = Label(window, text="Song: ", font=("Helvetica", 16))
song_name_label.pack(side="top", padx=10, pady=10)

command_label = Label(window, text="", font=("Helvetica", 14), fg="red")
command_label.pack(side="top", padx=10, pady=10)

def get_song_name():
    return browser.find_element(By.XPATH, "/html/body/div[4]/div/div[2]/div[3]/footer/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/span/a").text

def display_command(text):
    global command_text, last_command_time
    command_text = text
    command_label.config(text=command_text)
    last_command_time = time.time()

def update_command_label():
    global command_text, last_command_time
    if time.time() - last_command_time > 3:
        command_text = ""
        command_label.config(text="")
        song_name_label.config(text = get_song_name())
    window.after(100, update_command_label)

def update_camera():
    global last_swipe_time, previous_position, last_movement_time, hand_ready_for_swipe, last_fist_time, last_palm_time, fist_detected, palm_detected, hand_ready_for_gesture

    success, frame = cap.read()
    if success:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                index_finger_tip = hand_landmarks.landmark[12]
                current_position = (index_finger_tip.x, index_finger_tip.y)

                if previous_position is None:
                    previous_position = current_position

                if is_only_index_extended(hand_landmarks.landmark):
                    hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                    if hand_still:
                        hand_ready_for_swipe = True
                    if hand_ready_for_swipe:
                        swipe_direction, last_swipe_time = detect_swipe(previous_position, current_position, last_swipe_time)
                        if swipe_direction == 'left':
                            print("Swipe left detected: Previous track")
                            previous_track()
                            hand_ready_for_swipe = False
                            previous_position = current_position
                        elif swipe_direction == 'right':
                            print("Swipe right detected: Next track")
                            next_track()
                            hand_ready_for_swipe = False
                            previous_position = current_position

                if is_fist_closed(hand_landmarks.landmark):
                    hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                    
                    if hand_still:
                        hand_ready_for_gesture = True
                    if hand_ready_for_gesture and not fist_detected:
                        last_fist_time = time.time()
                        fist_detected = True
                    elif fist_detected and time.time() - last_fist_time > 1:
                        print("Fist closed and still for 1 second: Pausing")
                        pause_spotify()
                        hand_ready_for_gesture = False
                        fist_detected = False

                else:
                    fist_detected = False

                if is_palm_open(hand_landmarks.landmark):
                    hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                    
                    if hand_still:
                        hand_ready_for_gesture = True
                    if hand_ready_for_gesture and not palm_detected:
                        last_palm_time = time.time()
                        palm_detected = True
                    elif palm_detected and time.time() - last_palm_time > 1:
                        print("Open palm and still for 1 second: Playing")
                        play_spotify()
                        hand_ready_for_gesture = False 
                        palm_detected = False

                else:
                    palm_detected = False

                previous_position = current_position
        elif hand_ready_for_swipe or hand_ready_for_gesture:
            hand_ready_for_swipe = False
            hand_ready_for_gesture = False
        
        img = Image.fromarray(frame)
        imgtk = ImageTk.PhotoImage(image=img)
        camera_frame.imgtk = imgtk
        camera_frame.configure(image=imgtk)


    # Schedule the next frame update
    window.after(10, update_camera)

# Start updating camera and command label
update_camera()
update_command_label()

window.mainloop()
cap.release()


In [ ]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk
import time

# Global variables
is_video_playing = False
cap = None
webcam_cap = None
video_source = None

# Function to update the video frame in Tkinter window
def update_video():
    global is_video_playing, cap, webcam_cap

    if is_video_playing and cap.isOpened():
        ret, frame = cap.read()  # Capture frame from video
        if ret:
            # Resize the frame to fit in the 600x600 box
            frame = cv2.resize(frame, (600, 600))

            # Convert the frame to RGB (OpenCV uses BGR by default)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Convert to ImageTk format
            img = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            
            # Update the image on the label
            video_label.imgtk = imgtk  # Keep reference to avoid garbage collection
            video_label.config(image=imgtk)
        
        # If video ends, stop and switch back to live webcam
        if not ret:
            stop_video()
            start_webcam()
    
    elif not is_video_playing:
        # Update with webcam frames
        ret, frame = webcam_cap.read()
        if ret:
            # Resize the webcam frame to fit in the 600x600 box
            frame = cv2.resize(frame, (600, 600))

            # Convert webcam frame to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Convert to ImageTk format
            img = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            
            # Update the image on the label
            video_label.imgtk = imgtk  # Keep reference to avoid garbage collection
            video_label.config(image=imgtk)
    
    # Continue the video loop
    video_label.after(15, update_video)

# Play the tutorial video
def play_tutorial_video(tutorial_path):
    global is_video_playing, cap
    if not is_video_playing:
        stop_webcam()  # Stop webcam feed
    cap = cv2.VideoCapture(tutorial_path)  # Open the tutorial video
    if cap.isOpened():
        is_video_playing = True
        update_video()  # Start video update loop

# Pause the current video
def pause_video():
    global is_video_playing
    is_video_playing = False

# Stop the current video
def stop_video():
    global is_video_playing, cap
    is_video_playing = False
    if cap:
        cap.release()
        cap = None

# Start webcam capture
def start_webcam():
    global webcam_cap
    webcam_cap = cv2.VideoCapture(1)  # 0 for default webcam, 1 for secondary webcams
    if webcam_cap.isOpened():
        update_video()

# Stop webcam capture
def stop_webcam():
    global webcam_cap
    if webcam_cap:
        webcam_cap.release()
        webcam_cap = None

# Button functions to play tutorials
def play_play_tutorial():
    play_tutorial_video(r"Gesture_Video\play.mp4")

def play_pause_tutorial():
    play_tutorial_video(r"Gesture_Video\Pause.mp4")

def play_next_tutorial():
    play_tutorial_video(r"Gesture_Video\Next.mp4")

def play_prev_tutorial():
    play_tutorial_video(r"Gesture_Video\previous.mp4")

def play_volume_up_tutorial():
    play_tutorial_video(r"Gesture_Video\Volume up.mp4")

def play_volume_down_tutorial():
    play_tutorial_video(r"Gesture_Video\Volume down.mp4")

# Initialize the Tkinter window
root = tk.Tk()
root.title("Live Webcam Feed with Info Panel")
root.geometry("1000x600")  # Set window size

# Create a frame for the left side webcam feed
video_frame = tk.Frame(root, width=600, height=600)
video_frame.grid(row=0, column=0, padx=10, pady=10)

# Create a label to display the webcam/video
video_label = tk.Label(video_frame)
video_label.pack()

# Create a frame for the right side info panel and control buttons
control_frame = tk.Frame(root, width=400, height=600, bg="#f0f0f0")
control_frame.grid(row=0, column=1, padx=10, pady=10, sticky="nsew")

# Info Panel with label
info_label = tk.Label(control_frame, text="Gesture Control Tutorials", font=("Arial", 16), bg="#f0f0f0")
info_label.pack(pady=10)

# Control buttons for tutorials
play_button = tk.Button(control_frame, text="Play Tutorial", command=play_play_tutorial)
play_button.pack(pady=5)

pause_button = tk.Button(control_frame, text="Pause Tutorial", command=play_pause_tutorial)
pause_button.pack(pady=5)

next_button = tk.Button(control_frame, text="Next Track Tutorial", command=play_next_tutorial)
next_button.pack(pady=5)

prev_button = tk.Button(control_frame, text="Previous Track Tutorial", command=play_prev_tutorial)
prev_button.pack(pady=5)

volume_up_button = tk.Button(control_frame, text="Volume Up Tutorial", command=play_volume_up_tutorial)
volume_up_button.pack(pady=5)

volume_down_button = tk.Button(control_frame, text="Volume Down Tutorial", command=play_volume_down_tutorial)
volume_down_button.pack(pady=5)

# Start the webcam feed initially
start_webcam()

# Start the Tkinter event loop
root.mainloop()

# Release video capture when window is closed
if cap:
    cap.release()
if webcam_cap:
    webcam_cap.release()
